In [ ]:
%reload_ext autoreload
%autoreload 2

import time
import itertools
import numpy as np
import matplotlib.pyplot as plt

import tqdm 
import tqdm.notebook

from optimization import G1, G2
from optimization_clean import *
import multiprocessing


In [ ]:
def plotResult(selected_cities_n, loss_values, num_cities_per_step):
    print('Final loss ', loss_values[-1])
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    axes[0].plot(loss_values)
    axes[0].set_ylabel('Loss')
    axes[0].set_xlabel('Iterations')
    axes[0].set_title('Loss Evolution')
    
    
    m = selected_cities_n == 1
    not_selected = selected_cities_n == 0
    axes[1].scatter(g.x[not_selected, 0], g.x[not_selected, 1], s=8)
    axes[1].scatter(g.x[m, 0], g.x[m, 1], c='r', s=8)
    axes[1].set_title('Selected cities')
    axes[1].set_aspect(1)

    plt.xlim(0, 1)
    plt.ylim(0, 1)
    if num_cities_per_step is not None:
        plt.figure(figsize=(4, 2))
        plt.plot(np.arange(num_cities_per_step.shape[0]), num_cities_per_step)
        plt.title("#selected cities in each step")

In [ ]:
# Here we would load a given data set

np.random.seed(42)
N = 1000
g = G2(N)
# g = G1(N)

l = 0.8 # lambda

In [ ]:
# Run the optimization to compute the selected cities
n_iter = 10000
n_second_pass_iter = 5000

use_multithreading = True
total_runs = 12
use_kd_tree = True
import optimization

def beta_(i):
    p1 = 0.5
    p50 = 0.0001
    beta0 = -np.log(p1)
    beta1 = -np.log(p50)
    v = beta0 * (beta1 / beta0) ** (i/(n_iter-1.0))
    return v * 4


def beta(i):
    if i < n_iter // 2:
        return 0.4
    elif i < n_iter * 2 / 3:
        return 1
    else:
        return 5

def run_optimization(i):
    np.random.seed(i)

    if i == 0:
        curr_selected, curr_selected_convex, curr_loss_values, convex_loss = optimization.optimize(g, l, 
                                                                  beta=beta_, 
                                                                  n_iter=n_iter+n_second_pass_iter, 
                                                                  verbose=not use_multithreading, 
                                                                  initial_selection_probability=0.0,
                                                                  mutation_strategy=1)
        curr_selected[-1] = curr_selected_convex 
        curr_loss_values[-1] = convex_loss
        return curr_selected, curr_loss_values
    
    
    curr_selected, curr_loss_values = optimize(
        g, l, beta=beta, n_iter=n_iter, verbose=not use_multithreading, use_kd_tree=use_kd_tree)
    
    def beta2(i):
        if i < n_second_pass_iter // 2:
            return 5
        elif i < n_iter * 2 / 3:
            return 10
        else:
            return 20

    curr_selected2, curr_selected_convex, curr_loss_values2, convex_loss = optimization.optimize(g, l, 
                                                                  beta=10, 
                                                                  n_iter=n_second_pass_iter, 
                                                                  mutation_strategy=4,
                                                                  verbose=not use_multithreading, 
                                                                  selected_cities=curr_selected[-1])
    print(convex_loss, "vs old ", curr_loss_values[-1])
    curr_selected = np.concatenate([curr_selected, curr_selected2])
    curr_loss_values = np.concatenate([curr_loss_values, curr_loss_values2])
    curr_selected[-1] = curr_selected_convex 
    curr_loss_values[-1] = convex_loss
    return curr_selected, curr_loss_values

losses = []
all_selected = []
if use_multithreading:
    with multiprocessing.Pool(10) as p:
        with tqdm.notebook.tqdm(range(total_runs)) as pbar:
            for i, result in enumerate(p.imap_unordered(run_optimization, range(total_runs))):
                selected, loss_values = result
                pbar.update()
                losses.append(loss_values)
                all_selected.append(selected)
else:
    for i in tqdm.notebook.tqdm(range(total_runs)):
        selected, loss_values = run_optimization(i)
        losses.append(loss_values)
        all_selected.append(selected)

# Select the best solution
losses = np.array(losses)
plt.figure()
for i in range(total_runs):
    plt.plot(np.arange(losses[i].shape[0]), losses[i])
plt.title('Loss Values')
min_idx = np.argmin(losses[:, -1])
loss_values = losses[min_idx, :]
selected = all_selected[min_idx]

if len(selected) > 1:
    num_cities_per_step = np.array(selected).sum(axis=1)
else: 
    num_cities_per_step = None
selected = selected[-1]
plotResult(selected, loss_values, num_cities_per_step)